## Sample DB Source
[Sample SQLite Database Source](https://www.sqlitetutorial.net/sqlite-sample-database)

## DB Schema
![DB Schema](sqlite-sample-database-color.jpg)

In [1]:
import sqlalchemy

sqlalchemy.__version__

'1.4.39'

In [2]:
import pandas as pd
pd.__version__

'1.5.3'

In [3]:
from sqlalchemy import create_engine
# db+driver://<nohostname>/<path>
# where <path> is relative:
engine = create_engine("sqlite+pysqlite:///sqlitetutorial_sampledb.db")
# if we want internal logs to be displayed
# engine = create_engine("sqlite+pysqlite:///sqlitetutorial_sampledb.db", echo=True)

In [4]:
# https://docs.sqlalchemy.org/en/20/core/metadata.html
from sqlalchemy import MetaData

In [5]:
# we always start out with a collection that will be where we place our tables known as the MetaData object. 
# This object is essentially a facade around a Python dictionary that stores a series of Table objects 
# keyed to their string name
metadata = MetaData(engine)

In [6]:
# https://docs.sqlalchemy.org/en/20/orm/extensions/automap.html
# Define an extension to the sqlalchemy.ext.declarative system which automatically generates mapped classes and 
# relationships from a database schema, typically though not necessarily one which is reflected.
from sqlalchemy.ext.automap import automap_base

In [7]:
Base = automap_base()
Base.prepare(engine, reflect=True)

In [12]:
#import for handling exceptions
from sqlalchemy import exc

# execute within session
from sqlalchemy.orm import Session

In [13]:
# simple reading a table without join into a dataframe
try:
    with Session(engine) as session:
        albums = Base.classes.albums
        qry = session.query(albums)
        print('query> ' + str(qry.statement))
        albums_df = pd.read_sql(qry.statement, session.bind)
except exc.SQLAlchemyError as e:
    print(type(e), e)
    
albums_df.head()

query> SELECT albums."AlbumId", albums."Title", albums."ArtistId" 
FROM albums


,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [55]:
# joining two tables
def get_album_tracks(album_nm):
    try:
        with Session(engine) as session:
            albums = Base.classes.albums
            tracks = Base.classes.tracks
            qry = session.query(tracks.Name, tracks.Composer). \
                join(albums). \
                filter(albums.Title == album_nm)
            print('query> ' + str(qry.statement))
            tracks_df = pd.read_sql(qry.statement, session.bind)
            return tracks_df
    except exc.SQLAlchemyError as e:
        print(type(e), e)

In [56]:
get_album_tracks('Restless and Wild')

query> SELECT tracks."Name", tracks."Composer" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId" 
WHERE albums."Title" = :Title_1


,Name,Composer
0,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
1,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
2,Princess of the Dawn,Deaffy & R.A. Smith-Diesel


In [60]:
# joining three tables but with single link between them
def get_album_titles_genre(album_nm):
    try:
        with Session(engine) as session:
            albums = Base.classes.albums
            tracks = Base.classes.tracks
            genres = Base.classes.genres
            qry = session.query(albums.Title, tracks.Name, genres.Name). \
                join(albums,genres). \
                filter(albums.Title == album_nm)
            print('query> ' + str(qry.statement))
            tracks_df = pd.read_sql(qry.statement, session.bind)
            return tracks_df
    except exc.SQLAlchemyError as e:
        print(type(e), e)

In [62]:
get_album_titles_genre('Big Ones')

query> SELECT albums."Title", tracks."Name", genres."Name" AS "Name_1" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId" JOIN genres ON genres."GenreId" = tracks."GenreId" 
WHERE albums."Title" = :Title_1


,Title,Name,Name_1
0,Big Ones,Walk On Water,Rock
1,Big Ones,Love In An Elevator,Rock
2,Big Ones,Rag Doll,Rock
3,Big Ones,What It Takes,Rock
4,Big Ones,Dude (Looks Like A Lady),Rock
5,Big Ones,Janie's Got A Gun,Rock
6,Big Ones,Cryin',Rock
7,Big Ones,Amazing,Rock
8,Big Ones,Blind Man,Rock
9,Big Ones,Deuces Are Wild,Rock


In [93]:
#an example of join query and renmaing the column
def get_titles_genre(genre):
    try:
        with Session(engine) as session:
            albums = Base.classes.albums
            tracks = Base.classes.tracks
            genres = Base.classes.genres
            qry = session.query(tracks.Name.label('Song'), albums.Title, genres.Name.label('Genre')). \
                join(albums,genres). \
                filter(genres.Name == genre)
            print('query> ' + str(qry.statement))
            tracks_df = pd.read_sql(qry.statement, session.bind)
            return tracks_df
    except exc.SQLAlchemyError as e:
        print(type(e), e)

In [94]:
get_titles_genre('Drama')

query> SELECT tracks."Name" AS "Song", albums."Title", genres."Name" AS "Genre" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId" JOIN genres ON genres."GenreId" = tracks."GenreId" 
WHERE genres."Name" = :Name_1


,Song,Title,Genre
0,Don't Look Back,"Heroes, Season 1",Drama
1,One Giant Leap,"Heroes, Season 1",Drama
2,Collision,"Heroes, Season 1",Drama
3,Hiros,"Heroes, Season 1",Drama
4,Better Halves,"Heroes, Season 1",Drama
...,...,...,...
59,Something Nice Back Home,"LOST, Season 4",Drama
60,Cabin Fever,"LOST, Season 4",Drama
61,"There's No Place Like Home, Pt. 1","LOST, Season 4",Drama
62,"There's No Place Like Home, Pt. 2","LOST, Season 4",Drama


In [95]:
get_titles_genre('Opera')

query> SELECT tracks."Name" AS "Song", albums."Title", genres."Name" AS "Genre" 
FROM tracks JOIN albums ON albums."AlbumId" = tracks."AlbumId" JOIN genres ON genres."GenreId" = tracks."GenreId" 
WHERE genres."Name" = :Name_1


,Song,Title,Genre
0,"Die Zauberflöte, K.620: ""Der Hölle Rache Kocht...",Mozart Gala: Famous Arias,Opera


In [106]:
# an example of complex join with different joining than above (not really required here cause of single keys 
# but to demo)
def get_album_cust_info(album_nm):
    try:
        with Session(engine) as session:
            albums = Base.classes.albums
            tracks = Base.classes.tracks
            invoice_items = Base.classes.invoice_items
            invoices = Base.classes.invoices
            customers = Base.classes.customers
            qry = session.query(customers.City, customers.Country, customers.Email, 
                                tracks.Name.label('Track'), albums.Title.label('Album')). \
                join(invoices, customers.CustomerId == invoices.CustomerId). \
                join(invoice_items, invoices.InvoiceId == invoice_items.InvoiceId). \
                join(tracks, invoice_items.TrackId == tracks.TrackId). \
                join(albums, tracks.AlbumId == albums.AlbumId). \
                filter(albums.Title == album_nm)
            print('query> ' + str(qry.statement))
            tracks_df = pd.read_sql(qry.statement, session.bind)
            return tracks_df
    except exc.SQLAlchemyError as e:
        print(type(e), e)

In [109]:
get_album_cust_info('Greatest Hits')

query> SELECT customers."City", customers."Country", customers."Email", tracks."Name" AS "Track", albums."Title" AS "Album" 
FROM customers JOIN invoices ON customers."CustomerId" = invoices."CustomerId" JOIN invoice_items ON invoices."InvoiceId" = invoice_items."InvoiceId" JOIN tracks ON invoice_items."TrackId" = tracks."TrackId" JOIN albums ON tracks."AlbumId" = albums."AlbumId" 
WHERE albums."Title" = :Title_1


,City,Country,Email,Track,Album
0,Helsinki,Finland,terhi.hamalainen@apple.fi,Are You Gonna Go My Way,Greatest Hits
1,Brasília,Brazil,fernadaramos4@uol.com.br,Rock And Roll Is Dead,Greatest Hits
2,Helsinki,Finland,terhi.hamalainen@apple.fi,Mr. Cab Driver,Greatest Hits
3,London,United Kingdom,emma_jones@hotmail.com,Mr. Cab Driver,Greatest Hits
4,Yellowknife,Canada,ellie.sullivan@shaw.ca,American Woman,Greatest Hits
5,Brasília,Brazil,fernadaramos4@uol.com.br,I Belong To You,Greatest Hits
6,Helsinki,Finland,terhi.hamalainen@apple.fi,Believe,Greatest Hits
7,São Paulo,Brazil,eduardo@woodstock.com.br,Don't Look Back,Greatest Hits
8,Madrid,Spain,enrique_munoz@yahoo.es,Jah Seh No,Greatest Hits
9,Ottawa,Canada,edfrancis@yachoo.ca,Bush Doctor,Greatest Hits


In [146]:
from sqlalchemy import select
from sqlalchemy import func, desc

In [136]:
try:
    customers = Base.classes.customers
    stmt = select(customers.Country, func.count(customers.Country).label('Num_Customers')).group_by(customers.Country)
    print(stmt)
#    with engine.connect() as conn:
#        for row in conn.execute(stmt):
#            print(row)
    with Session(engine) as session:        
        result = session.execute(stmt).fetchall()
        print(result)
        print('----------------')
        for row in session.execute(stmt):
            print(row)
except exc.SQLAlchemyError as e:
        print(type(e), e)

SELECT customers."Country", count(customers."Country") AS "Num_Customers" 
FROM customers GROUP BY customers."Country"
[('Argentina', 1), ('Australia', 1), ('Austria', 1), ('Belgium', 1), ('Brazil', 5), ('Canada', 8), ('Chile', 1), ('Czech Republic', 2), ('Denmark', 1), ('Finland', 1), ('France', 5), ('Germany', 4), ('Hungary', 1), ('India', 2), ('Ireland', 1), ('Italy', 1), ('Netherlands', 1), ('Norway', 1), ('Poland', 1), ('Portugal', 2), ('Spain', 1), ('Sweden', 1), ('USA', 13), ('United Kingdom', 3)]
----------------
('Argentina', 1)
('Australia', 1)
('Austria', 1)
('Belgium', 1)
('Brazil', 5)
('Canada', 8)
('Chile', 1)
('Czech Republic', 2)
('Denmark', 1)
('Finland', 1)
('France', 5)
('Germany', 4)
('Hungary', 1)
('India', 2)
('Ireland', 1)
('Italy', 1)
('Netherlands', 1)
('Norway', 1)
('Poland', 1)
('Portugal', 2)
('Spain', 1)
('Sweden', 1)
('USA', 13)
('United Kingdom', 3)


In [147]:
try:
    albums = Base.classes.albums
    tracks = Base.classes.tracks
    invoice_items = Base.classes.invoice_items
    invoices = Base.classes.invoices
    customers = Base.classes.customers
    
    stmt = select(customers.Email, customers.Country, func.count(invoices.InvoiceId)). \
                    join_from(customers, invoices). \
                    group_by(customers.Country). \
                    order_by(desc(func.count(invoices.InvoiceId))). \
                    having(func.count(invoices.InvoiceId)>1)
    print(stmt)
    with Session(engine) as session:        
        print('----------------')
        for row in session.execute(stmt):
            print(row)
except exc.SQLAlchemyError as e:
        print(type(e), e)

SELECT customers."Email", customers."Country", count(invoices."InvoiceId") AS count_1 
FROM customers JOIN invoices ON customers."CustomerId" = invoices."CustomerId" GROUP BY customers."Country" 
HAVING count(invoices."InvoiceId") > :count_2 ORDER BY count(invoices."InvoiceId") DESC
----------------
('fharris@google.com', 'USA', 91)
('ftremblay@gmail.com', 'Canada', 56)
('camille.bernard@yahoo.fr', 'France', 35)
('luisg@embraer.com.br', 'Brazil', 35)
('leonekohler@surfeu.de', 'Germany', 28)
('emma_jones@hotmail.com', 'United Kingdom', 21)
('jfernandes@yahoo.pt', 'Portugal', 14)
('frantisekw@jetbrains.com', 'Czech Republic', 14)
('manoj.pareek@rediff.com', 'India', 13)
('joakim.johansson@yahoo.se', 'Sweden', 7)
('enrique_munoz@yahoo.es', 'Spain', 7)
('stanisław.wójcik@wp.pl', 'Poland', 7)
('bjorn.hansen@yahoo.no', 'Norway', 7)
('johavanderberg@yahoo.nl', 'Netherlands', 7)
('lucas.mancini@yahoo.it', 'Italy', 7)
('hughoreilly@apple.ie', 'Ireland', 7)
('ladislav_kovacs@apple.hu', 'Hungary'